# TensorTools Runtime Demo

Demonstrates the usage of the runtime using a simple autoencoder model.

In [1]:
# Force matplotlib to use inline rendering
%matplotlib inline

import os
import sys

# add path to libraries for ipython
sys.path.append(os.path.expanduser("~/libs"))

import numpy as np
import tensorflow as tf
import tensortools as tt

In [2]:
BATCH_SIZE = 24
REG_LAMBDA = 5e-4
NUM_GPUS = 2

In [3]:
dataset_train = tt.datasets.moving_mnist.MovingMNISTTrainDataset(BATCH_SIZE * NUM_GPUS,
                                                                 1)
dataset_valid = tt.datasets.moving_mnist.MovingMNISTValidDataset(BATCH_SIZE * NUM_GPUS,
                                                                 1)
#dataset_test = tt.datasets.moving_mnist.MovingMNISTTestDataset(BATCH_SIZE * NUM_GPUS,
#                                                               1)

File mnist.h5 has already been downloaded.
File mnist.h5 has already been downloaded.


In [4]:
class SimpleAutoencoderModel(tt.model.AbstractModel):
    """Simple neuronal autoencoder.
    """
    def __init__(self, inputs, targets, reg_lambda,
                 scope=None, is_training=False):
        self._scope = scope  # TODO: move to base class (call is device scope?)
        self._is_training = is_training  # TODO: move to base class
        super(SimpleAutoencoderModel, self).__init__(inputs, targets, reg_lambda)
    
    @tt.utils.attr.lazy_property
    def predictions(self):
        x = tf.contrib.layers.flatten(self._inputs)
        encoded = tt.network.fc("FC_Enc", x, 64,
                                weight_init=tf.contrib.layers.xavier_initializer(),
                                bias_init=0.0)
        representation = encoded
        decoded = tt.network.fc("FC_Dec", representation, self.input_shape[1] * self.input_shape[2] * self.input_shape[3],
                                weight_init=tf.contrib.layers.xavier_initializer(),
                                bias_init=0.0)
        
        return tf.reshape(decoded, [-1, 1, self.input_shape[1], self.input_shape[2], self.input_shape[3]])

    @tt.utils.attr.lazy_property
    def loss(self):
        return tt.loss.bce(self.predictions, self._targets)

In [5]:
runtime = tt.core.MultiGpuRuntime()
runtime.register_datasets(dataset_train, dataset_valid)
runtime.register_model(
    lambda x, y, scope, is_training : SimpleAutoencoderModel(x, y,
                                                             reg_lambda=REG_LAMBDA,
                                                             scope=scope,
                                                             is_training=is_training))
runtime.setup_feeding(tt.core.FEEDING_AUTOENCODER)
runtime.build()

In [6]:
runtime.train(epochs=5)

Starting epoch 1...
@    10: loss:     0.299, total-loss:     0.299 ( 4361.7 examples/sec,  0.01 sec/batch)
@    20: loss:     0.218, total-loss:     0.218 ( 4105.4 examples/sec,  0.01 sec/batch)
@    30: loss:     0.183, total-loss:     0.183 ( 5442.7 examples/sec,  0.01 sec/batch)
@    40: loss:     0.179, total-loss:     0.179 ( 4571.4 examples/sec,  0.01 sec/batch)
@    50: loss:     0.169, total-loss:     0.169 ( 4132.6 examples/sec,  0.01 sec/batch)
@    60: loss:     0.166, total-loss:     0.166 ( 4716.6 examples/sec,  0.01 sec/batch)
@    70: loss:     0.150, total-loss:     0.150 ( 3409.3 examples/sec,  0.01 sec/batch)
@    80: loss:     0.151, total-loss:     0.151 ( 4318.1 examples/sec,  0.01 sec/batch)
@    90: loss:     0.148, total-loss:     0.148 ( 3768.3 examples/sec,  0.01 sec/batch)
@   100: loss:     0.140, total-loss:     0.140 ( 4058.2 examples/sec,  0.01 sec/batch)

@   100: Starting validation (batch-size: 48, dataset-size: 10000):
9984/9984 [====================

In [ ]:
runtime.validate()

In [ ]:
runtime.test()

In [ ]:
runtime.close()